In [1]:
import pandas as pd 
import os
import json
import numpy as np
import ast
import nltk
import re
pd.options.display.max_columns=40
from sklearn.base import TransformerMixin , BaseEstimator

In [3]:
data = pd.read_csv(r'C:\Users\saadb\Desktop\dev\recommendation_project\jupyter_notebook\datasets\movies_cleaned.csv')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45400 entries, 0 to 45399
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45400 non-null  object 
 1   belongs_to_collection  3168 non-null   object 
 2   budget                 8888 non-null   float64
 3   genres                 45400 non-null  object 
 4   id                     45400 non-null  object 
 5   original_language      45386 non-null  object 
 6   original_title         45400 non-null  object 
 7   overview               44453 non-null  object 
 8   popularity             45394 non-null  float64
 9   production_companies   44910 non-null  object 
 10  production_countries   45389 non-null  object 
 11  release_date           45336 non-null  object 
 12  revenue                7408 non-null   float64
 13  runtime                43603 non-null  float64
 14  spoken_languages       45400 non-null  object 
 15  st

In [2]:
data = pd.read_csv(r'C:\Users\saadb\recommendation_project\recommendation_model\dataset\movies_metadata.csv',low_memory=False,date_parser='release_date')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\saadb\\recommendation_project\\recommendation_model\\dataset\\movies_metadata.csv'

In [ ]:
data.head(3)

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.isna().sum()

initial thoughts about data :-
-irrelevant columns ( id - imdb_id - homepage - poster_path ) id maybe useable if needed to merge data 
-columns ( belongs_to_collection - tagline ) have a lot of missing data inspect to see what is the reason and how to deal with it 

In [ ]:
#dropping irrelevant columns 
data.drop(['imdb_id','homepage','poster_path'],axis=1,inplace=True)

In [ ]:
#nothing suspicious about belongs to collection having so much nans 
data[data['belongs_to_collection'].isnull()].head()

In [ ]:
json.loads(data['belongs_to_collection'][0].replace("'",'"'))

In [ ]:
# function to handle json data replacing "" and '' as json format should be and then use json.loads and if anything went 
#wrong or some addidtion error like i have visuilized values like nan instead of json dict we replace those with np.nan
def handle_json(x):
    try :
        
        x = x.replace("'",'"')
    
        x = json.loads(x)
        
    except :
        return np.nan
    
    return x

In [ ]:
#apply handle_json to 'belongs_to_collection' 
data['belongs_to_collection'] = data['belongs_to_collection'].apply(lambda x : handle_json(x))

In [ ]:
#action may improve performance 
#notice that number of nan values increased i was expecting that number of nans will be equal to the number showen above from  
# info function after printing number of nan of all columns this number increased by 1323 need to investigate more in this 
data['belongs_to_collection'].isnull().value_counts()

In [ ]:
#extracting collection name from 'belongs_to_collection' as the useful information about this feature 
data['belongs_to_collection'].apply(lambda x : x['name'] if isinstance(x,dict) else np.nan)

In [ ]:
data['belongs_to_collection'] = data['belongs_to_collection'].apply(lambda x : x['name'] if isinstance(x,dict) else np.nan)

In [ ]:
# for genres we need to extract all possible genres from multiple dict same concept to deal with json format but to get the 
# useful final form of the feature will need to modify code used for extraction 
data['genres'][0]

In [ ]:
#dealng with json format 
data['genres'] = data['genres'].apply(lambda x : handle_json(x))

In [ ]:
# used code from course on udemy for the join idea 
data['genres'].apply(lambda x : '|'.join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [ ]:
data['genres'] = data['genres'].apply(lambda x : '|'.join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [ ]:
# for language column numbers are being counted as language and i think this is not valid 
data['original_language'].value_counts().tail(25)

# so they are string i think to get the numbers we can use nltk and use that to eliminate number like string or we can iterte 
# over every row in language column and chech if it contain number 

In [ ]:
x=[]
for ele in data['original_language'].dropna():
    results = re.findall('[0-9]+',ele)
    if results:
        x.append(results[0])

In [ ]:
x

In [ ]:
#so after extracting numbers that was parsed as strings using re we can remove them but they are represented in float format 
# and we captured integer format thats why '.0' is added to them 
for i in x:
    i = i + '.0'
    data['original_language'].replace(i,np.nan,inplace=True)

In [ ]:
#i need to see the duplicate if they are movies remake
data['original_title'].value_counts()

In [ ]:
#incpecting hamlet for example
hamlet = data[(data['original_title']=='Hamlet')]

In [ ]:
#as we see they are the same tilte but different movies so movie remade 
hamlet

In [ ]:
#here i can replace 'No overview found' , 'No Overview' , 'No movie overview available' and blancks with 'missing'
data['overview'].value_counts()

In [ ]:
strings_to_replace_overview = ['No overview found.' , 'No Overview' , 'No movie overview available', ' ','No movie overview available.']
for ele in strings_to_replace_overview:
    data['overview'].replace(ele,'missing',inplace=True)

In [ ]:
data['overview'].value_counts(ascending=False)

In [ ]:
data['popularity'].value_counts(ascending=False)

In [ ]:
data['popularity'].replace('Beware Of Frost Bites',np.nan,inplace=True)

In [ ]:
string_popularity_no_floats = [x for x in data['popularity'] if not isinstance(x,float)]

In [ ]:
string_popularity = [x for x in string_popularity_no_floats if re.search('[a-zA-Z]', x)]

In [ ]:
string_popularity

after reviwing popularity values noticed that we have 66 movie with 0.0 popularity and this seem odd somehow tried to view those movies but couldnt becuase the numbers are strings and cant convert them to floats becuase we have string also 'Beware Of Frost Bites' i captured by regular expressions so i need to remove or replace 'Beware Of Frost Bites' by nan and convert popularity type to float and inspect the odd popularities so after inspecting the 0.0 popularity thier is relationship i think between missing values in most of columns and the 0.0 popularity we can maybe remove or drop these rows.

In [ ]:
data['popularity'] = data['popularity'].astype(float)

In [ ]:
index_of_zero_popularity = data[(data['popularity']==0.0)].index

In [ ]:
data.drop(index_of_zero_popularity , axis=0 ,inplace=True)

In [ ]:
index_of_zero_popularity

In [ ]:
# from 'production_companies' column i only need name so i can try extract that 
data['production_companies'][1]

In [ ]:
#here missing values represented by empty list []
data['production_companies'].value_counts(ascending=False)

In [ ]:
data['production_companies'] = data['production_companies'].apply(lambda x : handle_json(x))

In [ ]:
data['production_companies'][1][0]['name']

In [ ]:
data['production_companies'] = data['production_companies'].apply(lambda x : '|'.join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [ ]:
data['production_countries'][0]

In [ ]:
data['production_countries'] = data['production_countries'].apply(lambda x : handle_json(x))

In [ ]:
data['production_countries'] = data['production_countries'].apply(lambda x : '|'.join(i['name'] for i in x) if isinstance(x,list) else np.nan)

In [ ]:
#missing values represented here by empty space we need to replace that with 'missing' 
data['production_countries'].value_counts(ascending=False)

In [ ]:
data['production_countries'].replace('' , 'missing', inplace=True)

In [ ]:
#for budget column we have a lot of values with 0 budget and this is not true or vlad value so we can replace these values with 
#np.nan
data['budget'].value_counts(ascending=False)

In [ ]:
#thier is not a lot of missing values crossponding to 0 budget so we can just replace them with np.nan
data[(data['budget']=='0')]

In [ ]:
#cupturing strings in budget column
string_budget = [x for x in data['budget'] if re.search('[a-zA-Z]', x)]

In [ ]:
string_budget

In [ ]:
for i in string_budget:
    data['budget'].replace(i,np.nan,inplace=True)

In [ ]:
data['budget'] = data['budget'].astype(float)

In [ ]:
data['budget'].replace(0.0,np.nan,inplace=True)

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'],errors='coerce')

In [ ]:
data['release_date'].value_counts(ascending=False)

In [ ]:
#ruffly same 0.0 as 'budget' for 'revenue' column as we dont know budget we cant know revenue i guess i will replace 0.0 with np.nan too
data['revenue'].value_counts(ascending=False)

In [ ]:
data['revenue'].replace(0.0,np.nan,inplace=True)

In [ ]:
# runtime for 1558 is 0.0 and this is not valid also so i will replace these with np.nan also 
data['runtime'].value_counts(ascending=False)

In [ ]:
data['runtime'].replace(0.0,np.nan,inplace=True)

In [ ]:
data['spoken_languages'].value_counts(ascending=False)

In [ ]:
data['spoken_languages'][1]

In [ ]:
data['spoken_languages'] = data['spoken_languages'].apply(lambda x : handle_json(x))

In [ ]:
data['spoken_languages'] = data['spoken_languages'].apply(lambda x : '|'.join(i['name'] for i in x) if isinstance(x,list) else 'missing')

In [ ]:
data['status'].value_counts(ascending=False)

In [ ]:
# for tagline i can replace - and blancks with missing 
data['tagline'].value_counts(ascending=False)

In [ ]:
tagline_replace = ['-' , '']
for i in tagline_replace:
    data['tagline'].replace(i,'missing',inplace=True)

In [ ]:
data['title'].value_counts(ascending=False)

In [ ]:
data['video'].value_counts(ascending=False)

In [ ]:
# vote average for 2998 0.0 !!
data['vote_average'].value_counts(ascending=False)

In [ ]:
# note 'genres' , 'production_companies' and 'spoken_languages' have blancks deal with this 
data[(data['vote_average']==0.0)].head(30)

In [ ]:
data['production_companies'].replace('','missing',inplace=True)
data['genres'].replace('','missing',inplace=True)
data['spoken_languages'].replace('','missing',inplace=True)

In [ ]:
#both vote average and vote count 0 to the same rows obersvations 
data.groupby(['vote_average'])['vote_count'].count()

inspect why vote average of these movies 0

In [ ]:
data['vote_count'].value_counts(ascending=False)

inspect why some movies have vote count 1

In [ ]:
data['budget'].isna().sum()

scale revenue and budget 

In [ ]:
data['budget'] = data['budget']/1000000

In [ ]:
data['revenue'] = data['revenue']/1000000

In [ ]:
os.chdir(r'C:\Users\saadb\recommendation_project')

In [ ]:
data.to_csv(path_or_buf='jupyter_notebook/datasets/movies_cleaned.csv',index=False)

In [ ]:
data.info()

In [ ]:
df = pd.read_csv('jupyter_notebook/datasets/movies_cleaned.csv')
df.head(3)